# Agents as Tools with Strands Agents SDK and Claude 4 Interleaved Thinking

This notebook demonstrates how to use Strands Agents SDK with Claude 4's **interleaved thinking** capability to orchestrate intelligent workflows with specialist agents.

## Understanding Interleaved Thinking

### What is Interleaved Thinking?

Interleaved thinking is a new capability in Claude 4 models that allows the model to:

1. **Think between tool calls**: Process and reason about results before deciding next steps
2. **Chain multiple tools with reasoning**: Make sophisticated multi-step decisions
3. **Adapt strategies dynamically**: Change approach based on intermediate results

### How It Works

There are a lot of similarities between Agent's event loop implemented with and without interleaved thinking:
```
Query → LLM is thinking -> LLM decides to call a Tool -> Event Loop calls the Tool -> Ouput is sent back to LLM -> [ this continues until LLM no longer needs to call any tools - it rendered the Final Answer ]
```

The main difference you'll notice with the interleaved thinking is that Event loop is acting on LLM's "thoughts", rather than "decisions". Notice the second link in the loop above, called "thinking". In a traditional event loop, the thoughts are hidden. We have to wait until LLM renders either a decision to call a tool or produces the Final Answer. 

In case of interleaved thinking, LLM is "leaking" its thoughts into the even loop while it's still in that second step - "LLM is thinking" - and event loop is configured to executed the tools as soon as LLM "thinks" about doing it. What this means is that by the time LLM is done thinking, it actually has the Final Answer, on the very first "decision". 


### Enabling Interleaved Thinking

To enable this feature with Strands and Bedrock:
- Set `temperature=1` (required when thinking is enabled)
- Add beta header: `"anthropic_beta": ["interleaved-thinking-2025-05-14"]`
- Configure reasoning budget: `"reasoning_config": {"type": "enabled", "budget_tokens": 3000}`

## Setup and Imports

In [1]:
%pip install strands-agents strands-agents-tools python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os
from strands import Agent, tool
from strands.models import BedrockModel
from strands.models import bedrock

# bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"

## Define Specialist Agents as Tools

First, we are going to create four specialist agents using the Strands `@tool` decorator:
- **Researcher**: Gathers factual information
- **Data Analyst**: Processes and analyzes information
- **Fact Checker**: Verifies information accuracy
- **Report Writer**: Creates polished final documents

In [4]:
from prompts import CODE_ANALYSIS_PROMPT
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import file_read

#  Specialist agents implemented as tools using Strands @tool decorator
@tool
def code_reader(repo_path: str) -> str:
    """
    Code reader specialist that can search through a repository and find relevant content.
    
    Args:
        repo_path: The repository path of the repository to analyze
        
    Returns:
        Reading findings
    """

    bedrock_model = BedrockModel(model_id="us.anthropic.claude-sonnet-4-20250514-v1:0")

    result = str()

    try:
        env = {"FASTMCP_LOG_LEVEL": "DEBUG",
        "AWS_PROFILE": "eldimi-Admin",
        "AWS_REGION": "us-east-1",
                    }

        git_repo_mcp_server = MCPClient(
            lambda: stdio_client(
                StdioServerParameters(
                    command="uvx",
                    args=["awslabs.git-repo-research-mcp-server@latest"],
                    env=env,
                )
            )
        )

        with git_repo_mcp_server:

            tools = [git_repo_mcp_server.list_tools_sync()]
            code_reader_agent = Agent(
                model=bedrock_model,
                system_prompt=CODE_ANALYSIS_PROMPT,
                tools=tools,
            )

            result = str(code_reader_agent(repo_path))
            print("\n\n")
    except Exception as e:
        return f"Error processing your query: {str(e)}"
    
    return result

In [5]:
@tool
def code_converter(data: str) -> str:
    """
    Converts PostreSQL analytics queries to ClickHouse analytics queries.
    
    Args:
        data: the file paths, code and code description
        
    Returns:
        The converted queries
    """
    # Analyst agent focuses on extracting insights
    code_converter_agent = Agent(
        model="",
        system_prompt="",
        callback_handler=None
    )
    
    # Analyze the provided data
    result = code_converter_agent(data)
    return str(result)

In [6]:
@tool
def code_writer(repo_path: str, coverted_code:str) -> str:
    """
    Writes new code in the repository given the provided coverted_code queries
    
    Args:
        repo_path: The path of the repository to write the code to
        coverted_code: the converted queries
        
    Returns:
        Fact-check results with accuracy assessment
    """
    code_writer_agent = Agent(
        model="",
        system_prompt="",
        callback_handler=None
    )
    
    # Verify the information
    result = code_writer_agent(f"Fact-check this information: {information}")
    return str(result)

## Claude 4 Orchestrator with Interleaved Thinking

Now we create the orchestrator - a Claude 4 agent that uses interleaved thinking to intelligently coordinate the specialist agents.

### How the Orchestrator Works:

1. Receives a high-level task from the user
2. **Thinks** about what information is needed
3. Calls the researcher tool to gather initial data
4. **Thinks** about the research results and what analysis is needed
5. Calls the data analyst to process findings
6. **Thinks** about accuracy and verification needs
7. May call the fact checker if needed
8. **Thinks** about how to present the findings
9. Calls the report writer for final output
10. **Reflects** on the complete workflow before responding

In [7]:
# Claude 4 Orchestrator with Interleaved Thinking using Strands
class StrandsInterlevedWorkflowOrchestrator:
    def __init__(self):
        # Define the orchestrator system prompt for intelligent workflow coordination
        self.system_prompt = """You are an intelligent workflow orchestrator with access to specialist agents.

        Your role is to intelligently coordinate a workflow using these specialist agents:
        - code_reader: Reads a repository content from the file system and searches for ALL postgreSQL analytics queries
        - code_converter: Converts the found postgres analytics queries to ClickHouse analytics queries
        - code_writer: Replaces the postgres analytics queries implementation with the new ClickHouse analytics queries  

        The agents will run sequentially code_reader -> code_converter -> code_writer. 
        The code_reader might need to run multiple times to identify ALL postgreSQL queries.
        The coordinator can re-try and validate accordingly.
        The coordinator understands the output of each agent and provides the output if needed as input to the next agent.
        """
    
    def run_workflow(self, task: str, enable_interleaved_thinking: bool = True) -> str:
        """Execute an intelligent workflow for the given task.
        
        Args:
            task: The task to complete
            enable_interleaved_thinking: Whether to enable interleaved thinking (default: True)
        
        The orchestrator will:
        1. Understand the task requirements
        2. Think about the best approach
        3. Coordinate specialist agents
        4. Reflect on results between steps
        5. Produce a comprehensive output
        """
        thinking_mode = "WITH interleaved thinking" if enable_interleaved_thinking else "WITHOUT interleaved thinking"
        print(f"\nStarting intelligent workflow {thinking_mode} for: {task}")
        print("=" * 70)
        
        # Configure Claude 4 with or without interleaved thinking via Bedrock
        if enable_interleaved_thinking:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1,  # Required to be 1 when thinking is enabled
                additional_request_fields={
                    # Enable interleaved thinking beta feature
                    "anthropic_beta": ["interleaved-thinking-2025-05-14"],
                    # Configure reasoning parameters
                    "reasoning_config": {
                        "type": "enabled",  # Turn on thinking
                        "budget_tokens": 3000  # Thinking token budget
                    }
                }
            )
        else:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1
            )
        
        # Create the orchestrator agent with Claude 4 and specialist tools
        orchestrator = Agent(
            model=claude4_model,
            system_prompt=self.system_prompt,
            tools=[code_reader, code_converter, code_writer]
        )
        
        prompt = f"""Complete this task using intelligent workflow coordination: {task}

        Instructions:
        1. Think carefully about what information you need to accomplish this task
        2. Use the specialist agents strategically - each has unique strengths
        3. After each tool use, reflect on the results and adapt your approach
        4. Coordinate multiple agents as needed for comprehensive results
        5. Ensure accuracy by fact-checking when appropriate
        6. Provide a comprehensive final response that addresses all aspects
        
        Remember: Your thinking between tool calls helps you make better decisions.
        Use it to plan, evaluate results, and adjust your strategy.
        """
        
        try:
            result = orchestrator(prompt)
            return str(result)
        except Exception as e:
            return f"Workflow failed: {e}"

## Run the Demo

Let's see the orchestrator in action! Watch how it thinking and making tool calling while it's thinking.


In [8]:
# Create the orchestrator
print("Strands Agents SDK: Claude 4 Interleaved Thinking Workflow Demo")
print("=" * 70)

try:
    orchestrator = StrandsInterlevedWorkflowOrchestrator()
    print("✅ Orchestrator initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize orchestrator: {e}")

Strands Agents SDK: Claude 4 Interleaved Thinking Workflow Demo
✅ Orchestrator initialized successfully!


In [ ]:
# Run the workflow with a test case
repo_path = "/Users/eldimi/Documents/ch/HouseClick"

print(f"📋 Repository path: {repo_path}\n")

try:
    result = orchestrator.run_workflow(repo_path)
    
    print(f"\n📊 Workflow Result:")
    print("=" * 70)
    print(result)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")

📋 Repository path: /Users/eldimi/Documents/ch/HouseClick


Starting intelligent workflow WITH interleaved thinking for: /Users/eldimi/Documents/ch/HouseClick
I need to complete a task that involves analyzing a repository at `/Users/eldimi/Documents/ch/HouseClick` and converting PostgreSQL analytics queries to ClickHouse analytics queries. Based on the available tools and workflow description, I need to:

1. Use code_reader to find all PostgreSQL analytics queries in the repository
2. Use code_converter to convert those queries to ClickHouse format
3. Use code_writer to replace the original queries with the converted ones

Let me start by using the code_reader to analyze the repository and find PostgreSQL queries.I'll help you convert PostgreSQL analytics queries to ClickHouse queries in your repository. Let me start by analyzing the repository structure and identifying all PostgreSQL queries.
Tool #1: code_reader
I'll help you identify ALL PostgreSQL analytics queries within the reposi

## Try without Interleaved Thinking

Experiment with calling the orchestrator and disabling interleaved thinking. Observe the difference in the output.

In [ ]:
# Now let's try the same task WITHOUT interleaved thinking
print("\n" + "="*70)
print("🔄 Now running the same task WITHOUT interleaved thinking")
print("="*70)

try:
    result_without_thinking = orchestrator.run_workflow(test_case, enable_interleaved_thinking=False)
    
    print(f"\n📊 Workflow Result (Without Interleaved Thinking):")
    print("=" * 70)
    print(result_without_thinking)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")